# Pydantic AI Demos - Part 1 (Structure)

This notebook demonstrates how pydantic AI supports returning results in a structured form. It starts with the basics
and then goes on with more complicated examples.

## Basic LLM Call

In this example we will just call an LLM and wait for an answer. We use the well known example of asking for the capital of a country.
This just returns a string.


In [2]:
from pydantic_ai.models.anthropic import AnthropicModel
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

BASE_URL="http://127.0.0.1:1234/v1"
LM_STUDIO_MODEL="openai/gpt-oss-20b"

model = OpenAIChatModel(LM_STUDIO_MODEL, provider=OpenAIProvider(BASE_URL))

agent = Agent(model, output_type=str)

result = await agent.run("What is the capital of Germany?")
result.output


'The capital of Germany is **Berlin**.'



## Some structure to the result

We would like some more information and we also would like it in another form than as string.

So lets create a target object and use this.


In [8]:
from pydantic import BaseModel, Field

class Capital(BaseModel):
    name: str = Field(..., description="The name of the capital")
    country: str = Field(..., description="The country of the capital, as it was asked by the user.")
    num_inhabitants: int = Field(..., description="The number of inhabitants of the capital")

agent = Agent(model, output_type=Capital)

result = await agent.run("What is the capital of Germany?")
hauptstadt = result.output
print(f"name: {hauptstadt.name}, country: {hauptstadt.country}, num_inhabitants: {hauptstadt.num_inhabitants}")

name: New Delhi, country: India, num_inhabitants: 21000000



## Lets add some logging

We are using logfire. A product of pydantic to see what happens during the call.

You would need to setup an account to get the token. There is a free account with a rather large limit.

In [9]:
from dotenv import load_dotenv
import logfire
import os

load_dotenv()
logfire.configure(token=os.getenv("LOGFIRE_TOKEN"))
logfire.info('I am configured!')
logfire.instrument_pydantic_ai()

07:42:05.252 I am configured!


Logfire project URL: 
]8;id=4223;https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos\https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos]8;;\


In [10]:
# Run the agent again.
result = await agent.run("What is the capital of Japan?")
result.output


07:43:00.186 agent run
07:43:00.198   chat openai/gpt-oss-20b


Capital(name='Tokyo', country='Japan', num_inhabitants=13929286)


## Add a validator

We not only want the result to be structured but also validate, that certain elements have a specific form.
Pydantic AI will do the validation for us and ask the LLM for correction if the result is wrong.
